In [4]:
import pandas as pd

# Load the data from the State Data CSV file into a DataFrame
state_data_df = pd.read_csv('StateDatabySeason63_49,48,62.csv')

# Display the initial DataFrame
print("Initial State Data DataFrame:")
print(state_data_df.head())


Initial State Data DataFrame:
  STATENAME                         URL                 WEBSITE  \
0   Alabama  http://adph.org/influenza/  Influenza Surveillance   
1   Alabama  http://adph.org/influenza/  Influenza Surveillance   
2   Alabama  http://adph.org/influenza/  Influenza Surveillance   
3   Alabama  http://adph.org/influenza/  Influenza Surveillance   
4   Alabama  http://adph.org/influenza/  Influenza Surveillance   

  ACTIVITY LEVEL ACTIVITY LEVEL LABEL      WEEKEND  WEEK   SEASON  
0        Level 1              Minimal  Oct-11-2008    41  2008-09  
1        Level 1              Minimal  Oct-18-2008    42  2008-09  
2        Level 1              Minimal  Oct-25-2008    43  2008-09  
3        Level 3              Minimal  Nov-01-2008    44  2008-09  
4        Level 1              Minimal  Nov-08-2008    45  2008-09  


In [10]:
# Columns to drop and the column to rename
state_data_drop_columns = ['URL', 'WEBSITE']  
state_data_rename_columns = {'STATENAME': 'States'}  

# Drop unnecessary columns
state_data_df = state_data_df.drop(columns=state_data_drop_columns, errors='ignore')

# Rename the column Statename
state_data_df = state_data_df.rename(columns=state_data_rename_columns)

# Display the cleaned DataFrame
print("Cleaned State Data DataFrame:")
print(state_data_df.head())


Cleaned State Data DataFrame:
    States ACTIVITY LEVEL ACTIVITY LEVEL LABEL    WEEKEND  WEEK   SEASON  YEAR
0  Alabama        Level 1              Minimal 2008-10-11    41  2008-09  2008
1  Alabama        Level 1              Minimal 2008-10-18    42  2008-09  2008
2  Alabama        Level 1              Minimal 2008-10-25    43  2008-09  2008
3  Alabama        Level 3              Minimal 2008-11-01    44  2008-09  2008
4  Alabama        Level 1              Minimal 2008-11-08    45  2008-09  2008


In [20]:
# Save the cleaned data to a new CSV file
state_data_df.to_csv('Cleaned_StateDatabySeason63_49,48,62.csv', index=False)

print("Cleaned data saved to 'Cleaned_StateDatabySeason63_49,48,62.csv'")


Cleaned data saved to 'Cleaned_StateDatabySeason63_49,48,62.csv'


In [13]:
import sqlite3

# Connect to SQLite database 
conn = sqlite3.connect('state_data.db')
cursor = conn.cursor()

# Define schema and create a table for the state data
cursor.execute('''
CREATE TABLE IF NOT EXISTS StateData (
    States TEXT,
    Activity_Level TEXT,
    Activity_Level_Label TEXT,
    Weekend DATE,
    Week INTEGER,
    Season TEXT
)
''')

# Insert cleaned state data into the database
state_data_df.to_sql('StateData', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()
